In [1]:
import torch
from torch import nn
import numpy as np
import sys
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import importlib

sys.path.append("..")
from model import BaselineTransformer
from data import get_database_path, read_h5_file
from utils import load_toml_config

import matplotlib.pyplot as plt


device = "cuda" if torch.cuda.is_available() else "mps" if sys.platform == "darwin" else "cpu"
random_seed = 114514
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [2]:
EMD_config = load_toml_config("EMD")
particle_type_scale = EMD_config['particle_type_scale']

# Load raw events

In [3]:
files = load_toml_config("file")
database_path = get_database_path()

events = {}
for key, value in files.items():
    events[key] = read_h5_file(database_path, value)
    print(key, events[key].shape)

SM (13451915, 19, 4)
neutral_boson (55969, 19, 4)
leptoquark (340544, 19, 4)
neutral_Higgs (691283, 19, 4)
charged_Higgs (760272, 19, 4)


In [4]:
signals = [key for key in events.keys() if key != "SM"]
print(signals)

['neutral_boson', 'leptoquark', 'neutral_Higgs', 'charged_Higgs']


In [5]:
model_hyper_parameters = load_toml_config("Transformer")
print(model_hyper_parameters)
feature_size = model_hyper_parameters["feature_size"]
embed_size = model_hyper_parameters["embed_size"]
num_heads = model_hyper_parameters["num_heads"]
num_layers = model_hyper_parameters["num_layers"]
hidden_dim = model_hyper_parameters["hidden_dim"]


classify_model = BaselineTransformer(feature_size, embed_size, num_heads, hidden_dim, num_layers)
classify_model.to(device)

{'feature_size': 3, 'embed_size': 16, 'num_heads': 8, 'num_layers': 4, 'hidden_dim': 256, 'output_dim': 4}


/home/desmondhe/anaconda3/envs/ad/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


BaselineTransformer(
  (particle_embedding): Linear(in_features=3, out_features=16, bias=True)
  (pos_encoder): ParticlePositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
        )
        (linear1): Linear(in_features=16, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=16, bias=True)
        (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=304, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

# Split Events

In [6]:
from analysis import create_exp_bkg_events, train_test_split, get_dataloaders
from data import EventDataset

In [11]:
test_ratio=0.2
val_ratio = 0.2
sig_lambda = 0.5
n = 100000

# test_signal = "neutral_boson"
# test_signal = "leptoquark"
# test_signal = "neutral_Higgs"
test_signal = "charged_Higgs"

exp_events, bkg_events = create_exp_bkg_events(events['SM'], events[test_signal], sig_lambda, n)
X1, X2, W1, W2 = train_test_split(exp_events, bkg_events, test_ratio)
n1 = len(W1)
m1 = len(X1)
pi = n1 / (n1 + m1)
n2 = len(W2)
m2 = len(X2)
train_dataloader, val_dataloader = get_dataloaders(X1[:,:,:3], W1[:, :, :3], val_ratio, )

# Train

In [12]:
from metrics import BinaryACCUpdater
classify_model.to(device)
optimizer = torch.optim.Adam(classify_model.parameters(), lr=1e-3, weight_decay=1e-5)
loss_fn = nn.BCELoss()
acc_metric = BinaryACCUpdater()
metric_dict = {"Accuracy": acc_metric}

In [13]:
from classify import train_model

In [14]:
train_model(
    classify_model, optimizer,
    loss_fn, metrics_dict=metric_dict,
    train_dataloader=train_dataloader, val_dataloader=val_dataloader,
    monitor="val_Accuracy", mode="max",
    epochs=50
    )


================================================================================2024-05-31 20:57:37
Epoch 1 / 50

100%|██████████| 125/125 [00:00<00:00, 353.94it/s, val_Accuracy=0.682, val_loss=0.58] 
<<<<<< reach best val_Accuracy : 0.6817812919616699 >>>>>>

================================================================================2024-05-31 20:57:51
Epoch 2 / 50

100%|██████████| 125/125 [00:00<00:00, 354.81it/s, val_Accuracy=0.676, val_loss=0.588]

================================================================================2024-05-31 20:58:06
Epoch 3 / 50

100%|██████████| 125/125 [00:00<00:00, 352.31it/s, val_Accuracy=0.687, val_loss=0.572]
<<<<<< reach best val_Accuracy : 0.6866875290870667 >>>>>>

================================================================================2024-05-31 20:58:20
Epoch 4 / 50

100%|██████████| 125/125 [00:00<00:00, 265.14it/s, val_Accuracy=0.685, val_loss=0.575]

=========================================================================

,train_loss,train_Accuracy,val_loss,val_Accuracy,epoch
0,0.589603,0.675500,0.579704,0.681781,1
1,0.579434,0.683906,0.588434,0.675813,2
2,0.578238,0.683984,0.572195,0.686688,3
3,0.576355,0.687398,0.574973,0.685250,4
4,0.575338,0.687922,0.570307,0.691031,5
5,0.574660,0.689305,0.571775,0.688531,6
6,0.573528,0.689289,0.569963,0.689688,7
7,0.574256,0.688422,0.569651,0.689469,8
8,0.573476,0.689406,0.569919,0.690094,9
9,0.572826,0.689305,0.572640,0.692500,10
